In [1]:
import os
import sys
import pandas as pd
import argparse
from datetime import datetime, timedelta
import pathlib

# 新增：將上一層目錄加入 sys.path
parent_path = pathlib.Path().absolute().parent
sys.path.insert(0, str(parent_path))
# Setup Django environment
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'website_configs.settings')
import django
django.setup()
# 重要：設定環境變數以允許在 Jupyter 的異步環境中執行同步操作
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

# Now we can import Django models
from app_user_keyword_db.models import NewsData

In [2]:
from django.db.models import Q, Max, F
from collections import Counter

In [3]:
# Searching keywords from "content" column
# This function now uses database queries instead of pandas
def filter_database_fullText(user_keywords, cond, cate, weeks):
    # Get the latest date in the database
    latest_date = NewsData.objects.aggregate(max_date=Max('date'))['max_date']
    
    # Calculate start date
    start_date = latest_date - timedelta(weeks=weeks)
    
    # Base query - filter by date range
    queryset = NewsData.objects.filter(date__gte=start_date, date__lte=latest_date)
    
    # Filter by category if not "全部"
    if cate != "全部":
        queryset = queryset.filter(category=cate)
    
    # Filter by keywords based on condition (AND or OR)
    if cond == 'and':
        # For AND condition, we need all keywords to be present
        for kw in user_keywords:
            queryset = queryset.filter(content__contains=kw)
    elif cond == 'or':
        # For OR condition, any keyword can be present
        q_objects = Q()
        for kw in user_keywords:
            q_objects |= Q(content__contains=kw)
        queryset = queryset.filter(q_objects)
    
    return queryset

In [6]:

user_keywords = ['車禍','意外']  # Example keywords
cond = 'or'  # Example condition (and/or), 
cate = '全部'  # Example category (or "全部" for all categories)
weeks = 4  # Example weeks
queryset = filter_database_fullText(user_keywords, cond, cate, weeks)

In [7]:
for news in queryset:
    print(news.title)

台中氣爆各縣市繃緊神經! 啟動百貨消防公安稽查
台中氣爆各縣市繃緊神經! 啟動百貨消防公安稽查
員警被控路口撞騎士肇逃 警方澄清未發生碰撞
慈濟關係企業工安意外成植物人 家屬陳情盼負起長照責任
中崙工業區電鍍廠發生工安意外 11人吸入性嗆傷送醫
新／豪宅建案工安意外！工人自高處墜落 送醫搶救中
台中工安意外！ 6工人清洗汙水槽 ＂疑缺氧＂有人摔落槽體
苗栗三義工安意外 1名工人被埋3米深邊坡
烤漆工廠工安意外！ 洗3尺深污水槽釀6人送醫｜#鏡新聞
快訊／台南電鍍廠工安意外！「聚氯化鋁混到漂白水」毒暈11人 1男不治
疑為人員疏失致災 台南電鍍廠工安意外釀1死10傷
台中市工安意外！ 25歲工人墜落鋼架 頭部重創送醫
長耕國際烤漆廠工安意外 「放流槽清淤」6人暈倒
高雄阿蓮工地意外 廢鐵砸頭釀1死 (圖)
台南驚悚工安意外！工人修遮光罩「6樓高墜下」雙腿變形
苗栗三義工安意外 1名工人被埋3米深邊坡
台中烤漆廠工安意外！6工人受困污水槽「2人昏迷」 疑一氧化碳超標釀禍
快訊／台中烤漆廠驚傳工安意外！6工人清洗槽體出狀況…2人昏迷送醫
台塑工安意外1人炸成「碎片」！原定2天後家庭旅行
高雄阿蓮區工地意外事故 一名女工人不幸身亡
烤漆廠工安意外 疑缺氧6人不適送醫
烤漆廠工安意外 疑缺氧6人不適送醫
快訊／北市錦州街新建案工安意外 55歲男墜樓送醫


In [8]:
# Limit to k results and get specific fields
news_items = queryset.values('category', 'title', 'link', 'photo_link')[:3]

In [9]:
news_items

<QuerySet [{'category': '北市事故', 'title': '台中氣爆各縣市繃緊神經! 啟動百貨消防公安稽查', 'link': 'https://tw.news.yahoo.com/%E5%8F%B0%E4%B8%AD%E6%B0%A3%E7%88%86%E5%90%84%E7%B8%A3%E5%B8%82%E7%B9%83%E7%B7%8A%E7%A5%9E%E7%B6%93-%E5%95%9F%E5%8B%95%E7%99%BE%E8%B2%A8%E6%B6%88%E9%98%B2%E5%85%AC%E5%AE%89%E7%A8%BD%E6%9F%A5-102236029.html', 'photo_link': 'https://s.yimg.com/lo/api/res/1.2/PB1VutNL9J0E987CcpPewA--~A/YXBwaWQ9dHdhYnVuZXdzO3c9MjIwO2g9MTI4O2ZpPWZpbGw7cHhvZmY9NTA7cHlvZmY9MQ--/https://media.zenfs.com/ko/ftvn.com.tw/3047d4e7a9287210171bd11a6e7fd0f8'}, {'category': '北市事故', 'title': '台中氣爆各縣市繃緊神經! 啟動百貨消防公安稽查', 'link': 'https://tw.news.yahoo.com/%E5%8F%B0%E4%B8%AD%E6%B0%A3%E7%88%86%E5%90%84%E7%B8%A3%E5%B8%82%E7%B9%83%E7%B7%8A%E7%A5%9E%E7%B6%93-%E5%95%9F%E5%8B%95%E7%99%BE%E8%B2%A8%E6%B6%88%E9%98%B2%E5%85%AC%E5%AE%89%E7%A8%BD%E6%9F%A5-105503587.html', 'photo_link': 'https://s.yimg.com/lo/api/res/1.2/f1ukmfI5hIBBnSNPA02Dtw--~A/YXBwaWQ9dHdhYnVuZXdzO3c9MjIwO2g9MTI4O2ZpPWZpbGw7cHhvZmY9NTA7cHlvZmY9MQ--/https://media.ze

In [10]:
dates = list(queryset.values_list('date', flat=True))

In [12]:
NewsData.objects.filter(category='北市事故').values('date')

<QuerySet [{'date': datetime.date(2025, 4, 4)}, {'date': datetime.date(2025, 4, 4)}, {'date': datetime.date(2025, 4, 4)}, {'date': datetime.date(2025, 4, 4)}, {'date': datetime.date(2025, 4, 4)}, {'date': datetime.date(2025, 4, 4)}, {'date': datetime.date(2025, 4, 4)}, {'date': datetime.date(2025, 4, 4)}, {'date': datetime.date(2025, 4, 4)}, {'date': datetime.date(2025, 4, 4)}]>